In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.metrics import *

In [ ]:
!pip install fastai

In [ ]:
path = Path('/kaggle/input/best-artworks-of-all-time/images/images/')

In [ ]:
src = ImageList.from_folder(path).split_by_rand_pct(0.2).label_from_folder()

In [ ]:
src

In [ ]:
tfms = get_transforms()

In [ ]:
data = (src.transform(tfms, size=112).databunch(bs=32).normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(5,5))

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate,model_dir="/tmp/model/")

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, max_lr=slice(1e-3,1e-2))

In [ ]:
learn.save('stage1')

In [ ]:
learn.load('stage1')

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=slice(1e-6,1e-5))

In [ ]:
learn.save('stage-2')

In [ ]:
learn.freeze()

In [ ]:
data2 = (src.transform(tfms, size=224).databunch(bs=32).normalize(imagenet_stats))

In [ ]:
learn.data=data2

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, max_lr=slice(1e-4,1e-3))

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=slice(1e-4,1e-3))

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(10, max_lr=slice(1e-6,1e-5))

In [ ]:
learn.show_results()

In [ ]:
learn.save('stage3')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_confusion_matrix(figsize=(20,20))

In [ ]:
interp.plot_top_losses(9,heatmap=True)